<span style="font-size:20px; font-weight:bold;">Tutorial 2: Brightway </span>

Since the explanation of brightway from scratch exceeds the framework of this repository, a few usefull sources for brightway beginners are listed below:

* Brightway official documentation: https://docs.brightway.dev/en/latest/#
* Documentation Brightway Tutorials: https://wiki.ubc.ca/Documentation:Brightway_Tutorials
* DdS Brightway Schools: https://www.d-d-s.ch/
* Brightcon Conference: https://docs.brightway.dev/en/latest/content/contact/contact.html

However, the setup and most important ideas of utilizing brightway for this framework are discussed in the following.

<span style="font-size:16px; font-weight:bold;">2.1 Setup in console or terminal</span>

After installing brightway according to the official documentation (https://docs.brightway.dev/en/latest/content/installation/index.html), we can check if the installment was succesfull through the console or terminal window. For example, if you are using conda for your python installation (please note that the exclamation mark is specific to Jupyter Notebook): 

In [1]:
!conda env list


# conda environments:
#
MyEnv                  C:\Users\jkueng\.conda\envs\MyEnv
myEnv                  C:\Users\jkueng\.conda\envs\myEnv
base                 * C:\Users\jkueng\AppData\Local\anaconda3
brightway              C:\Users\jkueng\AppData\Local\anaconda3\envs\brightway



Since the brightway environment appears in this list, the installation was successfull. In order to work with the brightway framework, we can select the environment by the following command:

In [5]:
!conda activate brightway

<span style="font-size:16px; font-weight:bold;">2.2 Setup of project and databases in code</span>

Now we can switch from the console or terminal, respectively, to the actual code file. The next step is to important the required libraries to work with brightway as well as the library pandas for data handling.

In [1]:
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi

import pandas as pd

Before diving into brightway, its structure needs to be emphasized. For a new LCA, one creates a project, which stores all the data, Life Cycle Impact Assessment (LCIA) methods, caluclations, etc. in different types of data objects. More information can be found here: https://docs.brightway.dev/en/latest/content/overview/structure.html

To create a new project with basic data on elementary flows and LCIA, the following command can be used:

In [ ]:
#bi.remote.install_project('ecoinvent-3.9.1-biosphere', 'project_E2DT')

Note that here different types and versions of databases such as ecoinvent, forwast, USEEIO can be used.

Executing this code is required only once, as the project is stored by brightway. The project can be selected afterwards like this:

In [2]:
selected_project = "project_E2DT"
bd.projects.set_current(name=selected_project)
print("Current project: ", bd.projects.current)

Current project:  project_E2DT


Since we included some basic data on elementary flows and LCIA through the ecoinvent database, we already discussed a major functionality of brightway: 
<span style="font-weight:bold;">databases. </span> They contain background and foreground information for the LCA, which you are using subsequently to build the Life Cycle Inventory (LCI).

To implement additional databases, e.g., those provided by ecoinvent or customized by LCA practitioners from an Excel file, brightway offers an import function: https://docs.brightway.dev/en/latest/content/cheatsheet/importing.html

Another option is to create a database from scratch and populate it within the code. Note that this enables additional flexibility compared to working with static Excel files, since we can modify the database dynamically. We will go into more detail on this later. For now, an empty database can be created like this:

In [3]:
del bd.databases["MEA_Carbon_Capture"]
my_database = bd.Database("MEA_Carbon_Capture")
my_database.register()

print("Databases loaded: ")
print(list(bd.databases))

db_biosphere3 = bd.Database("biosphere3")
db_ecoinvent = bd.Database("ecoinvent_cutoff_391")

Databases loaded: 
['biosphere3', 'ecoinvent_cutoff_391', 'MEA_Carbon_Capture']


It can be seen that there are three databases. While biosphere3 is the database providing basic data on elementary flows and LCIA, ecoinvent_cutoff_391 was implemented by using the importer functionality. In this way, we have a more activities from ecoinvent (e. g. the production of steel for the columns) available. The thrid database, MEA_Carbon_Capture, is our empty database, in which we generate customized activities for the purpose of the LCA in the next step.

<span style="font-size:16px; font-weight:bold;">2.3 Preparation for populating MEA_Carbon_Capture database</span>

<span style="font-weight:bold;">2.3.1 Including process simulation results</span>

To import process simulation results from Aspen Plus an exemplary Excel file Inventory_Simulation1.xlsx was prepared, in which the LCI based on one simulation is provided. Such a file can be generated as shown in Tutorial 1: Aspen-Python-Interface. Of course, there is the option do directly link the Aspen-Python-Interface with the brightway code. We will do that in the final framework. For now, we work with an Excel sheet and we can access it like this:

In [4]:
# Get Process Parameters of LCI
df_ProcessParameters_names = ["Solvent flow rate", "Solvent MEA conc. (LEANMEA)", "MEA prod. normalisation"]
df_ProcessParameters = pd.read_excel("Inventory_Simulation1.xlsx", names=df_ProcessParameters_names, usecols=[2,3,6],
                                         skiprows=10, nrows=1 )

# Get Material Flows of LCI
df_MaterialFlow_names =["CO2 captured", "FLUEGAS total", "FLUEGAS CO2", "FLUEGAS N2", "FLUEGAS O2", "FLUEGAS H2O", "MAKEUP Water",
                "MAKEUP Amine", "FLUEOFF total", "FLUEOFF CO2", "FLUEOFF N2", "FLUEOFF H2O", "FLUEOFF MEA", "WASHWAT in",
                "WATOUT out water", "WATOUT out MEA"]
df_MaterialFlow = pd.read_excel("Inventory_Simulation1.xlsx", names=df_MaterialFlow_names, usecols = list(range(2,18)),
                                         skiprows=20, nrows=1 )

# Get Natural Resources of LCI
df_NaturalResources_names = ["Cooling water COOLER", "Cooling water COND"]
df_NaturalResources = pd.read_excel("Inventory_Simulation1.xlsx", names=df_NaturalResources_names,usecols = [2,3],
                                          skiprows=30, nrows=1 )

# Get Energy flow of LCI
df_EnergyFlow_names = ["Reboiler steam in MJ/ton CO2", "Total electr.", "Pump electr.", "Reboiler steam in kWh", "Total energy"]
df_EnergyFlow = pd.read_excel("Inventory_Simulation1.xlsx", names=df_EnergyFlow_names,usecols = [2,3,4,5,6],
                                          skiprows=39, nrows=1 )

# Get Equipment and Infrastructure of LCI
df_equipment_names = ["Pumps", "Reboiler", "Compressor", "Condenser", "Cooler"]
df_equipment = pd.read_excel("Inventory_Simulation1.xlsx", names=df_equipment_names, usecols = [2, 3, 4, 5, 6],
                                          skiprows=47, nrows=1 )
df_Infrastructure_names = ["Absorber Column", "Absorber Packing", "Desorber Column", "Desorber Packing", "Norm Carbon capture plant"]
df_Infrastructure = pd.read_excel("Inventory_Simulation1.xlsx", names=df_Infrastructure_names,usecols = [3,4,5,6,9],
                                          skiprows=55, nrows=1 )

# For example: Print process parameters
print("Process parameters of LCI for simulation 1: \n")
print(df_ProcessParameters)

Process parameters of LCI for simulation 1: 

   Solvent flow rate  Solvent MEA conc. (LEANMEA)  MEA prod. normalisation
0              19300                           30                 0.000126


Of course, the import of data can be coded more general and efficient, but for didactic reasons, it is shown here in this way. 

<span style="font-weight:bold;">2.3.2 Including information from biosphere3 and ecoinvent databases</span>

In addition to the data from the simulation, we need some background information based on the ecoinvent_cutoff_391 database (provided by brightway's importer function from ecoinvent). Unfortunately, it is a tedious task to find specific activities in the database. For example, we can search for an acitivity to describe the steel produced for the columns of absorber and stripper like this:

In [5]:
print(db_ecoinvent.search("Market for steel, unalloyed"))

['market for steel, unalloyed' (kilogram, GLO, None), 'market for power sawing, with catalytic converter' (hour, GLO, None), 'market for assembly of generator and motor, auxilliaries and energy use, for mini CHP plant' (unit, GLO, None)]


The search returns several activities that contain the phrase we were looking for. The first activity seems most suitable for our purposes. Each activity is specified by several key words:

In [6]:
activity_steel = db_ecoinvent.search("market for steel, unalloyed")[0].as_dict()

for key in activity_steel.keys():
    print(key)

comment
classifications
activity type
activity
database
filename
location
name
synonyms
parameters
authors
type
reference product
flow
unit
production amount
code
id


It is recommended to always work with the "code" key of an acitivity, as it is a unique value. With this procedure, all required activities from the biosphere3 and ecoinvent database are obtained:

In [7]:
# Data (activities) from database biosphere 3 for Carbon dioxide, Nitrogen, Oxygen, Water, Monoethanolamine
biosphere3_act_CarbonDioxide_air = db_biosphere3.get(code="349b29d1-3e58-4c66-98b9-9d1a076efd2e")

biosphere3_act_CarbonDioxide_airurban = db_biosphere3.get(code="aa7cac3a-3625-41d4-bc54-33e2cf11ec46")
biosphere3_act_Nitrogen_air = db_biosphere3.get(code="e5ea66ee-28e2-4e9b-9a25-4414551d821c")
biosphere3_act_Oxygen_air = db_biosphere3.get(code="9ec076d9-6d9f-4a0b-9851-730626ed4319")
biosphere3_act_Water_airurban = db_biosphere3.get(code="5d368100-b1bc-4456-8420-e469edccf349")
biosphere3_act_Water_air = db_biosphere3.get(code="075e433b-4be4-448e-9510-9a5029c1ce94")
biosphere3_act_Monoethanolamine_air = db_biosphere3.get(code="9f550c00-0f0f-45e7-a29c-a450752d4c7a")

#Data (activities) from database ecoinvent:

# Market for steel, low-alloyed and unalloy for packings (low-alloyed) and columns (unalloyed)
ecoinvent_act_low_alloyed = db_ecoinvent.get(code="a81ce0e882f1b0ef617462fc8e7472e4")
ecoinvent_act_unalloyed = db_ecoinvent.get(code="d872e0d78319cb13e12b96de83e19dd7")

# Reboiler: heat production, natural gas, at industrial furnace >100kW
ecoinvent_act_heatprod = db_ecoinvent.get(code="b74c8625e0adf3b76b6a1aafed10312b")

# Market for monoethanolamine
ecoinvent_act_marketMEA = db_ecoinvent.get(code="1d95a9865e5ecb049eb64f6823865078")

# Market for tap water
ecoinvent_act_markettapwater = db_ecoinvent.get(code="4fe1f2dae4830593ee2d608cb2d5ff2c")

# Market for waste water, average
ecoinvent_act_marketwastewater = db_ecoinvent.get(code="ef9e2b6a0815008d49a77388e7c5b0e8")

# Market group for electricity, medium voltage
ecoinvent_act_marketelectr = db_ecoinvent.get(code="476b68dc744251f288055e5ce8264feb")

# Market for water, deionised
ecoinvent_act_marketwaterdeion = db_ecoinvent.get(code="95d26245bc411de42cbca001406f6131")

# Water production, deionsed
ecoinvent_act_waterproddeion = db_ecoinvent.get(code="395d665bd9b8ce065b590bd96fd74ca9")

# Market for air compressor, screw-type compressor, 300kW
ecoinvent_act_compr = db_ecoinvent.get(code="fdabbf27abc2d302066ebce8affec56a")

# Market for borehole heat exchanger, 150m
ecoinvent_act_heatex = db_ecoinvent.get(code="5d68a7fd42b989d214867478b999a04c")

# Market for gas boiler
ecoinvent_act_gasboil = db_ecoinvent.get(code="976694b4c0b31641b846a89d2fbf6c4b")

# Market for pump, 40W
ecoinvent_act_pump = db_ecoinvent.get(code="1358cde069b0d7df0f29529d19c6f900")

After importing the data, we want to use them to populate our customized database MEA_Carbon_Capture.

<span style="font-size:16px; font-weight:bold;">2.4 Populating MEA_Carbon_Capture database by generating activities and exchanges</span>

To provide some theoretical background, Brightway is based on the theory of computational structure for LCA. If a clean separation between products and processes is given, the technosphere matrix A can be directly modeled by activities (or nodes) for the processes and exchanges (or edges) for the products. However, Brightway modified this notation over the years, since many databases it builds on and utilises do not exhibit a clean separation between processes and products. This means that an activity can be both process and product and is therefore referenced in both technosphere rows and columns, leading to an abstraction of the original approach. For more information, the following references are recommended:
* Brightway documentation - Inventory data: https://docs.brightway.dev/en/latest/content/overview/inventory.html
* R. Heijungs and S. Suh, The computational structure of life cycle assessment (Ecoefficiency
in industry and science). Dordrecht: Kluwer, 2002.
* B. Steubing, C. Mutel, F. Suter, et al., “Streamlining scenario analysis and optimization of key choices in value chains using a modular LCA approach,” The International Journal of Life Cycle Assessment, vol. 21, no. 4, pp. 510–522, 2016. DOI: 10.1007/s11367-015-1015-3.

It is assumed that LCA practitioners are familiar with the procedure of generating the LCI. However, to facilitate starting with computational LCA, some advices for the workflow are provided:
* Based on Brightway, an open-sourve GUI called Activity Browser (AB) was developed recently: https://doi.org/10.1016/j.simpa.2019.100012
* Please note you have to be careful with respect to the versions of Brightway. Up to now, AB is only compatible with Brightway 2.0.
* However, working with the GUI facilitates the design of the LCA, especially in the initial phase when defining its goal and scope.
* A typical workflow could be to first create the LCI in AB and then extract the structure (activities and exchanges) to an Excel file. Then, this structure can be used to create its own brightway project, which is fully customizable.
* Later on, AB can still be used to visualize the LCA results of the created brightway code.

Based on this workflow, several activities are defined in the following:
  

In [8]:
######################################################################################################################
# DEFINE ACTIVITIES OF DATABASE MEA CARBON CAPTURE
######################################################################################################################

# Activity 1: Absorber
data_abs = {
    'name': 'Absorber',
    'code': '1',
    'location': 'GLO',
    'reference product': 'Absorber with Packing',
    'type': 'process',
    'unit': 'unit',
}
act_abs = my_database.new_activity(**data_abs)
act_abs.save()

# Activity 2: Desorber
data_des = {
    'name': 'Desorber',
    'code': '2',
    'location': 'GLO',
    'reference product': "Desorber with Packing",
    'type': "process",
    'unit': 'unit',
}
act_des = my_database.new_activity(**data_des)
act_des.save()

# Activity 3: MEA production
data_MEAprod = {
    'name': 'MEA production',
    'code': '4',
    'location': 'GLO',
    'reference product': "MEA production",
    'type': "process",
    'unit': 'kg',
}
act_MEAprod = my_database.new_activity(**data_MEAprod)
act_MEAprod.save()

# Activity 4:Infrastructure Absorption/ Desorption
data_InfraAbsDes = {
    'name': 'inf_absorption_desorption',
    'code': '5',
    'location': 'GLO',
    'reference product': 'Carbon Capture Plant',
    'type': 'process',
    'unit': 'unit',
}
act_InfraAbsDes = my_database.new_activity(**data_InfraAbsDes)
act_InfraAbsDes.save()

# Activity 5: Production Absorption/ Desorption
data_ProAbsDes = {
    'name': 'pro_absorption_desorption',
    'code': '6',
    'location': 'GLO',
    'reference product': 'CO2 produced',
    'type': 'process',
    'unit': 'kg/h',
}
act_ProAbsDes = my_database.new_activity(**data_ProAbsDes)
act_ProAbsDes.save()

For each activity, there are several exchanges either with one other activity from the custom database MEA_Carbon_Capture or one activity from the biosphere3 or ecoinvent database. We discuss the topic by taking a closer look at the exchanges for the activity "Production Absorption/ Desorption." The exchanges of this activity is particularly interesing, as it combines data from several databases and is used later to define the functional unit.

In [9]:
######################################## Exchanges for activity ProAbsDes (A5) #########################################

# A5 Exchange 1: Carbon dioxide, fossil
act_ProAbsDes.new_exchange(
    name = "Carbon dioxide, fossil",
    input = ("biosphere3",f"{biosphere3_act_CarbonDioxide_airurban['code']}"),
    amount = df_MaterialFlow.at[0, "FLUEOFF CO2"],
    unit = "kilogram",
    type = "biosphere",
).save()

# A5 Exchange 2: Monoethanolamine
act_ProAbsDes.new_exchange(
    name = "Monoethanolamine",
    input = ("biosphere3",f"{biosphere3_act_Monoethanolamine_air['code']}"),
    amount = df_MaterialFlow.at[0, "FLUEOFF MEA"],
    unit = "kilogram",
    type = "biosphere",
).save()

# A5 Exchange 3: Nitrogen
act_ProAbsDes.new_exchange(
    name = "Nitrogen",
    input = ("biosphere3",f"{biosphere3_act_Nitrogen_air['code']}"),
    amount = df_MaterialFlow.at[0, "FLUEOFF N2"],
    unit = "kilogram",
    type = "biosphere",
).save()

# A5 Exchange 4: Water
act_ProAbsDes.new_exchange(
    name = "Water",
    input = ("biosphere3",f"{biosphere3_act_Water_air['code']}"),
    amount = df_MaterialFlow.at[0, "FLUEOFF H2O"],
    unit = "cubicmeter",
    type = "biosphere",
).save()

# A5 Exchange 5: ProAbsDes
act_ProAbsDes.new_exchange(
    name = "pro_absorption_desorption",
    input = act_ProAbsDes,
    amount = df_MaterialFlow.at[0, "CO2 captured"],
    unit = "kilogram",
    type = "production",
).save()

# A5 Exchange 6: MEA production - Required MEA solvent
act_ProAbsDes.new_exchange(
    name = "MEA production",
    input = act_MEAprod,
    amount = df_ProcessParameters.at[0, "MEA prod. normalisation"],
    #amount=df_ProcessParameters.at[0, "Solvent flow rate"],
    unit = "kilogram",
    type = "technosphere",
).save()

# A5 Exchange 7: Reboiler - heat production, natural gas, at industrial furnace >100kW - Required Production of heat for steam generation in reboiler
act_ProAbsDes.new_exchange(
    name = "heat production, natural gas, at industrial furnace >100kW",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_heatprod['code']}"),
    amount = df_EnergyFlow.at[0, "Reboiler steam in MJ/ton CO2"] * (df_MaterialFlow.at[0,"CO2 captured"]/1000),
    unit = "megajoule",
    type = "technosphere",
).save()

# A5 Exchange 8: InfraAbsDes - Carbon capture plant - Required Infrastructure including columns, pumps, compressors, etc.
act_ProAbsDes.new_exchange(
    name = "Infrastructure Absorber/ Desorber",
    input = act_InfraAbsDes,
    amount = df_Infrastructure.at[0, "Norm Carbon capture plant"],
    #amount = 1,
    unit = "unit",
    type = "technosphere",
).save()

# A5 Exchange 9: Market for monoethanolamine - Produced MEA in wastewater
act_ProAbsDes.new_exchange(
    name = "Market for monoethanolamine",
    input =("ecoinvent_cutoff_391",f"{ecoinvent_act_marketMEA['code']}"),
    amount = df_MaterialFlow.at[0, "WATOUT out MEA"],
    unit = "kilogram",
    type = "technosphere",
).save()

# A5 Exchange 10: Market for tap water - Required Water for cooling
act_ProAbsDes.new_exchange(
    name = "Market for tap water",
    input =("ecoinvent_cutoff_391",f"{ecoinvent_act_markettapwater['code']}"),
    amount = df_NaturalResources.at[0, "Cooling water COOLER"] + df_NaturalResources.at[0, "Cooling water COND"],
    unit = "kilogram",
    type = "technosphere",
).save()

# A5 Exchange 11: Market for wastewater, average -  Produced Wastewater
act_ProAbsDes.new_exchange(
    name = "Market for tap wastewater, average",
    input =("ecoinvent_cutoff_391",f"{ecoinvent_act_marketwastewater['code']}"),
    amount = df_MaterialFlow.at[0, "WATOUT out water"],
    unit = "cubicmeter",
    type = "technosphere",
).save()

# A5 Exchange 12: Market group for electricity, medium voltage -  Required electricity for pumps, compressors, etc.
act_ProAbsDes.new_exchange(
    name = "Market group for electricity, medium voltage",
    input =("ecoinvent_cutoff_391",f"{ecoinvent_act_marketelectr['code']}"),
    amount = df_EnergyFlow.at[0, "Total electr."],
    unit = "kilowatt hour",
    type = "technosphere",
).save()

# A5 Exchange 13: water production, deionised - Required Washwater
act_ProAbsDes.new_exchange(
    name = "Water production, deionised",
    input =("ecoinvent_cutoff_391",f"{ecoinvent_act_waterproddeion['code']}"),
    amount = df_MaterialFlow.at[0, "WASHWAT in"],
    unit = "kilogram",
    type = "technosphere",
).save()

<span style="font-size:18px; font-weight:bold;">Wastewater Treatment Notes</span>

Wastewater handling requires special care because its **inventory entry can represent either a burden or a credit**, depending on how it’s modeled.

**Define the boundary clearly**

- **Burden →** the system *consumes* wastewater treatment (off-site service).  
- **Credit →** the system *exports* a treatment service or by-product that replaces a reference process.

**Choose the dataset carefully**

- Match the treatment process to the **real stream composition** (COD/TOC, N, amines, salts).  
- Avoid generic municipal averages for **industrial amine-rich** or **high-COD** streams.  
- If proxies are used, document and justify them.

**Check the sign**

- **Credits** should appear *negative*.  
- **Burdens** should appear *positive*.  


By analysing the exchanges one by one it can be seen that exchanges (E) 1-4 are related to emissions of relevant components. Thus, corresponding inputs are required from the biosphere3 database. Regarding the amount of each component, the results from the simulation stored in the dataframes are inserted. Then, there are several other exchanges E 6-13, which are <span style="font-weight:bold;">required</span> as an input for the process or <span style="font-weight:bold;">produced</span> as a by-product. Typically, there is also an exchange with the activity itself to describe its product. In this case, this means the captured CO<sub>2</sub> by the process (E5). Analogous, the exchanges for all the other activities are definied:

In [10]:
########################################################################################################################
# CREATE EXCHANGES FOR EACH ACTIVITY AND SET VALUES IN MEA CARBON CAPTURE DATABASE
########################################################################################################################

############################################# Exchanges for activity Absorber ##########################################

# Exchange 1: Absorber
act_abs.new_exchange(
    name = "Absorber",
    input = act_abs,
    amount = 1,
    unit = "unit",
    type = "production",
).save()

# Exchange 2: market for steel, low-alloyed
act_abs.new_exchange(
    name = "market for steel, low-alloyed",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_low_alloyed['code']}"),
    amount = df_Infrastructure.at[0, "Absorber Packing"],
    unit = "kilogram/kilogram CO2",
    type = "technosphere",
).save()

# Exchange 3: market for steel, unalloyed
act_abs.new_exchange(
    name = "market for steel, unalloyed",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_unalloyed['code']}"),
    amount = df_Infrastructure.at[0, "Absorber Column"],
    unit = "kilogram/kilogram CO2",
    type = "technosphere",
).save()

########################################### Exchanges for activity Desorber ############################################

# Exchange 1: Desorber
act_des.new_exchange(
    name = "Desorber",
    input = act_des,
    amount = 1,
    unit = "unit",
    type = "production",
).save()

# Exchange 2: market for steel, low-alloyed
act_des.new_exchange(
    name = "market for steel, low-alloyed",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_low_alloyed['code']}"),
    amount = df_Infrastructure.at[0, "Desorber Packing"],
    unit = "kilogram/kilogram CO2",
    type = "technosphere",
).save()

# Exchange 3: market for steel, unalloyed
act_des.new_exchange(
    name = "market for steel, unalloyed",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_unalloyed['code']}"),
    amount = df_Infrastructure.at[0, "Desorber Column"],
    unit = "kilogram/kilogram CO2",
    type = "technosphere",
).save()

######################################## Exchanges for activity MEA production #########################################

# Exchange 1: MEA production
act_MEAprod.new_exchange(
    name = "MEA production",
    input = act_MEAprod,
    amount = 1,
    unit = "kilogram",
    type = "production",
).save()

# Exchange 2: market for monoethanolamine
act_MEAprod.new_exchange(
    name = "market for monoethanolamine",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_marketMEA['code']}"),
    amount = 0.3,
    unit = "kilogram",
    type = "technosphere",
).save()

# Exchange 3: market for water, deionised
act_MEAprod.new_exchange(
    name = "market for water, deionised",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_marketwaterdeion['code']}"),
    amount = 0.7,
    unit = "kilogram",
    type = "technosphere",
).save()

######################################## Exchanges for activity InfraAbsDes #########################################

# Exchange 1: InfraAbsDes
act_InfraAbsDes.new_exchange(
    name = "InfraAbsDes",
    input = act_InfraAbsDes,
    amount = 1,
    unit = "unit",
    type = "production",
).save()

# Exchange 2: Absorber
act_InfraAbsDes.new_exchange(
    name = "Absorber",
    input = act_abs,
    amount = 1,
    unit = "unit",
    type = "technosphere",
).save()

# Exchange 3: Desorber
act_InfraAbsDes.new_exchange(
    name = "Desorber",
    input = act_des,
    amount = 1,
    unit = "unit",
    type = "technosphere",
).save()

# Exchange 4: Market for air compressor, screw-type compressor, 300kW
act_InfraAbsDes.new_exchange(
    name = "market for air compressor, screw-type compressor, 300kW",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_compr['code']}"),
    amount = 1,
    unit = "unit",
    type = "technosphere",
).save()

# Exchange 5: Market for borehole heat exchanger, 150m
act_InfraAbsDes.new_exchange(
    name = "Market for borehole heat exchanger, 150m",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_heatex['code']}"),
    amount = 3,
    unit = "unit",
    type = "technosphere",
).save()

# Exchange 6: Market for gas boiler
act_InfraAbsDes.new_exchange(
    name = "Market for gas boiler",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_gasboil['code']}"),
    amount = 1,
    unit = "unit",
    type = "technosphere",
).save()

# Exchange 7: Market for pump, 40W
act_InfraAbsDes.new_exchange(
    name = "Market for pump, 40W",
    input = ("ecoinvent_cutoff_391",f"{ecoinvent_act_pump['code']}"),
    amount = 6,
    unit = "unit",
    type = "technosphere",
).save()

<span style="font-size:16px; font-weight:bold;">2.5 LCA - Goal & scope</span>

In line with the LCA methodology (ISO 14040/44), we begin by defining the **goal and scope**, because these choices govern all subsequent modelling and interpretation. Here, the goal is to quantify the environmental impacts of a post-combustion carbon-capture process.

**Functional unit (FU).** Because performance is assessed on capture and associated to an efficiency (95%), the FU is the **amount of CO<sub>2</sub> treated**. In practice we set the FU to a convenient reference, and in Brightway we implement it by **demanding the activity `ProAbsDes`** with the corresponding mass of CO<sub>2</sub> (e.g., 1000 kg). This FU normalises all inventory and impact results and enables fair comparison across design or operating alternatives.

**LCIA method.** For impact assessment we select **ReCiPe 2016 v1.03 (midpoints)** including, for example, **climate change (GWP100)**. These methods are available in Brightway by default; if needed, **custom LCIA methods** can also be defined to match project-specific conventions.

In [11]:
# Define functional units (reference flow)
functional_unit = act_ProAbsDes

# List functional_units and their amount for LCA calculation (you could define several functional units)
list_functional_units = [
    {functional_unit: 1000}
]

# Find methods which will be used for LCIA, here: ReCiPe 2016 v1.03, endpoint (E) ( - no LT)
ReCiPe_methods = [
     method for method in bd.methods
     if "ReCiPe 2016 v1.03, midpoint (E)" in str(method) and 'LT' not in str(method)
]

Let's take a look on which indicators are included in this method. In addition, the units of all methods are stored in a list for later.

In [13]:
# Trim entries of ReCiPe method for listing
ReCiPe_methods_trim = [tup[1:] for tup in ReCiPe_methods]
print("ReCiPe 2016 v1.03, midpoints (E): \n")
for method in ReCiPe_methods_trim:
    print(method)

# Get units of each ReCiPe method
ReCiPe_units = [bd.Method(method).metadata['unit'] for method in ReCiPe_methods]

ReCiPe 2016 v1.03, midpoints (E): 

('acidification: terrestrial', 'terrestrial acidification potential (TAP)')
('climate change', 'global warming potential (GWP1000)')
('ecotoxicity: freshwater', 'freshwater ecotoxicity potential (FETP)')
('ecotoxicity: marine', 'marine ecotoxicity potential (METP)')
('ecotoxicity: terrestrial', 'terrestrial ecotoxicity potential (TETP)')
('energy resources: non-renewable, fossil', 'fossil fuel potential (FFP)')
('eutrophication: freshwater', 'freshwater eutrophication potential (FEP)')
('eutrophication: marine', 'marine eutrophication potential (MEP)')
('human toxicity: carcinogenic', 'human toxicity potential (HTPc)')
('human toxicity: non-carcinogenic', 'human toxicity potential (HTPnc)')
('ionising radiation', 'ionising radiation potential (IRP)')
('land use', 'agricultural land occupation (LOP)')
('material resources: metals/minerals', 'surplus ore potential (SOP)')
('ozone depletion', 'ozone depletion potential (ODPinfinite)')
('particulate matt

<span style="font-size:16px; font-weight:bold;">LCA - Life Cycle Impact Assessment</span>

With **goal & scope** defined (and the inventory model in place), the second step of our LCA workflow is **Life Cycle Impact Assessment (LCIA)**: mapping quantified flows to impact indicators consistent with the chosen method.

**Calculation setup.** Specify the **functional unit(s)** and the **LCIA method(s)** to be applied.

**Computation.** Use `multi_lca(...)` to evaluate the selected method in one pass and return all desired indicators (e.g., the ReCiPe 2016 v1.03 midpoints), reported per the stated FU(s). This yields the LCA results.

**Post-processing.** The remaining code only organizes and visualizes those results—structuring tables, ranking indicators, and plotting contributions—to support interpretation.

In [14]:
# Calculation setput
bd.calculation_setups["cal1"] = {
    'inv': list_functional_units,
    'ia': ReCiPe_methods
}

# Calculate LCA results
mlca = bc.multi_lca.MultiLCA("cal1")
LCA_results = mlca.results[0]

# Editing results for printing / extracting
ReCiPe_methods_trim = [tup[2:] for tup in ReCiPe_methods]
LCA_results = {
    method: {"Value": round(value, 2), "Unit": unit}
    for method, value, unit in zip(ReCiPe_methods_trim, LCA_results, ReCiPe_units)
}

# To dataframe
df_LCA_results = pd.DataFrame.from_dict(LCA_results, orient="index")
df_LCA_results.reset_index(names="Method", inplace=True)
print("ReCiPe 2016 v1.03, midpoints (E) - Calculation results: \n")
print(df_LCA_results)

ReCiPe 2016 v1.03, midpoints (E) - Calculation results: 

                                               Method     Value           Unit
0           terrestrial acidification potential (TAP)      0.07      kg SO2-Eq
1                  global warming potential (GWP1000)    248.14      kg CO2-Eq
2             freshwater ecotoxicity potential (FETP)     -4.35  kg 1,4-DCB-Eq
3                 marine ecotoxicity potential (METP) -44535.71  kg 1,4-DCB-Eq
4            terrestrial ecotoxicity potential (TETP)   -224.34  kg 1,4-DCB-Eq
5                         fossil fuel potential (FFP)    105.29      kg oil-Eq
6           freshwater eutrophication potential (FEP)     -0.36        kg P-Eq
7               marine eutrophication potential (MEP)     -1.38        kg N-Eq
8                     human toxicity potential (HTPc)   2647.51  kg 1,4-DCB-Eq
9                    human toxicity potential (HTPnc) -40530.75  kg 1,4-DCB-Eq
10                 ionising radiation potential (IRP)     13.29    kg Co-

<span style="font-size:16px; font-weight:bold;">LCA - Life Cycle Interpretation</span>

**Indicator selection**

**What we analyse (midpoints) — consistent with the E2DT article:**

* **Climate change (GWP100):** Primary objective of capture; directly reflects the heat/electricity burden of solvent regeneration.
* **Terrestrial acidification (TAP):** Sensitive to **SOₓ/NOₓ** from the energy supply (steam/electricity) that dominates operating impacts.
* **Photochemical oxidant formation (POFP):** Captures **NOₓ/VOC** emissions linked to the same energy drivers; often moves with acidification but is not redundant. 
* **Human toxicity (non-cancer, HTPnc):** Screens potential solvent-/amine-related pathways and wastewater handling effects. 
* **Freshwater & marine ecotoxicity (FETP, METP):** Track downstream releases (e.g., wash water management), useful when tuning wash sections and water balances. 

These six were used in the ## ARTICLE runs and capture the **energy-driven** (reboiler duty, electricity) and **emissions-to-water/air** mechanisms that actually move with our process knobs; they therefore give orthogonal, interpretable signals for scenario/design comparison.

**Practical note:**
Before exporting to Excel, we **filter indicators** to the subset above and drop categories that are **consistently ~0 or dominated** across all simulations—this keeps figures/tables readable without losing signal. The export then follows the same writer pattern as in Tutorial 1.

If a sensitivity or a new heat-supply scenario shows a filtered category becoming non-negligible, just re-enable it in the `methods` list and re-run `multi_lca`—the pipeline is agnostic to which ReCiPe midpoints you keep. 

In [19]:
df_LCA_results_filtered = df_LCA_results.iloc[[0, 1, 2, 3, 7, 9]]
print(df_LCA_results_filtered)

with pd.ExcelWriter(r"LCA_results.xlsx", mode="w" , engine="openpyxl") as writer:
    df_LCA_results_filtered.to_excel(writer, header=True, sheet_name="LCA_results", startrow=0, startcol=0)

                                      Method     Value           Unit
0  terrestrial acidification potential (TAP)      0.07      kg SO2-Eq
1         global warming potential (GWP1000)    248.14      kg CO2-Eq
2    freshwater ecotoxicity potential (FETP)     -4.35  kg 1,4-DCB-Eq
3        marine ecotoxicity potential (METP) -44535.71  kg 1,4-DCB-Eq
7      marine eutrophication potential (MEP)     -1.38        kg N-Eq
9           human toxicity potential (HTPnc) -40530.75  kg 1,4-DCB-Eq


**Normalization (for Visualization Only)**

To visualize indicators with different units and magnitudes, we apply **display normalization** — not methodological weighting.

Each indicator \( I \) is normalized across all runs as:

   I' = ((I<sub>max</sub> - I) / (I<sub>max</sub>  - I<sub>min</sub> )) × 100

- This transformation is **only used for plotting**, **not** for decision-making.  
- Always interpret results using the **raw, unit-bearing midpoint values**, not normalized indices.